<a href="https://colab.research.google.com/github/profteachkids/StemUnleashed/blob/main/Sphere_Area_Jacobian_GaussLaw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import jax.numpy as jnp
import jax
from jax.config import config
config.update("jax_enable_x64", True)
from plotly.subplots import make_subplots

In [2]:
def spherexyz(phi,theta,r):
    rxy = r*jnp.sin(theta)
    return jnp.array([rxy*jnp.cos(phi), rxy*jnp.sin(phi), r*jnp.cos(theta)])


In [3]:
spherexyz_vec = jnp.vectorize(spherexyz,signature='(),(),()->(3)')

In [4]:
sphere_jac=jax.jacobian(spherexyz,(0,1))

In [5]:
sphere_jac(0.,np.pi/2.,1.)

(DeviceArray([0., 1., 0.], dtype=float64, weak_type=True),
 DeviceArray([ 6.123234e-17,  0.000000e+00, -1.000000e+00],            dtype=float64, weak_type=True))

In [45]:
sphere_jaccross(np.pi/2,np.pi/2,1.)

DeviceArray([6.123234e-17, 1.000000e+00, 6.123234e-17], dtype=float64)

In [46]:
def sphere_jaccross(phi,theta,r):
    return -jnp.cross(*sphere_jac(phi,theta,r))

In [47]:
def sphere_jacdet(phi,theta,r):
    return jnp.linalg.norm(sphere_jaccross(phi,theta,r))

In [48]:
sphere_jacdet_vec=jnp.vectorize(sphere_jacdet,signature='(),(),()->()')

In [66]:
nphi=360
ntheta=360
phi=jnp.linspace(0,2*np.pi,nphi,endpoint=False)
theta=jnp.linspace(0,np.pi,ntheta,endpoint=False)

dphi=2*np.pi/(nphi)
dtheta=np.pi/(ntheta)
da = dphi*dtheta

jnp.sum(sphere_jacdet_vec(phi[None,:],theta[:,None],1.))*da

DeviceArray(12.56629087, dtype=float64)

In [67]:
4*jnp.pi

12.566370614359172

In [68]:
phi=jnp.linspace(0,2*np.pi,32)
theta=jnp.linspace(0,np.pi,32)
s=spherexyz_vec(phi[None,:],theta[:,None],1.)

In [69]:
fig = make_subplots(rows=1,cols=1,specs=[[{'type': 'surface'}]])
for i in range(theta.size):
    x,y,z = s[i].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='white')

for i in range(phi.size):
    x,y,z = s[:,i].T
    fig.add_scatter3d(x=x,y=y,z=z,mode='lines', line_color='white')

fig.update_layout(width=800,template='plotly_dark',showlegend=False)

In [75]:
e0=8.85e-12
def e_field(q,r, r0):
    d=r-r0
    return q/(4*np.pi*e0) * d/(jnp.linalg.norm(d)**3)

In [78]:
def integrand_dphi_dtheta(q, phi,theta,r, r0):
    n = sphere_jaccross(phi,theta,r)
    return jnp.dot(n,e_field(q, spherexyz(phi,theta,r),r0))

In [79]:
r0=jnp.array([0.5,0.,0.])
integrand_dphi_dtheta(1.,0.,np.pi/2,1.,r0)

DeviceArray(3.59672188e+10, dtype=float64)

In [80]:
integrand_dphi_dtheta_vec=jnp.vectorize(integrand_dphi_dtheta,signature='(),(),(),(),(3)->()')

In [81]:
nphi=360
ntheta=360
phi=jnp.linspace(0,2*np.pi,nphi,endpoint=False)
theta=jnp.linspace(0,np.pi,ntheta,endpoint=False)

dphi=2*np.pi/nphi
dtheta=np.pi/ntheta
da = dphi*dtheta

e0*jnp.sum(integrand_dphi_dtheta_vec(1.,phi[None,:],theta[:,None],1.,r0))*da

DeviceArray(0.99999546, dtype=float64)